## Shopify Internship Challenge

### Question 1

Let's import some of the necessary libraries to start our analysis.

In [2]:
import zipfile
import pandas as pd
from pandas.errors import EmptyDataError
import matplotlib.pyplot as plt 
import seaborn as sns

Import from Google spreadsheets to a dataframe.

In [3]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pickle
import os.path
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
def gsheet_api_check(SCOPES):
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

In [5]:
from googleapiclient.discovery import build
def pull_sheet_data(SCOPES,SPREADSHEET_ID,DATA_TO_PULL):
    creds = gsheet_api_check(SCOPES)
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=DATA_TO_PULL).execute()
    values = result.get('values', [])
    
    if not values:
        print('No data found.')
    else:
        rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                  range=DATA_TO_PULL).execute()
        data = rows.get('values')
        print("COMPLETE: Data copied")
        return data

In [9]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SPREADSHEET_ID = '16i38oonuX1y1g7C_UAmiK9GkY7cS-64DfiDMNiR41LM'
DATA_TO_PULL = 'Sheet1'
data = pull_sheet_data(SCOPES,SPREADSHEET_ID,DATA_TO_PULL)
df = pd.DataFrame(data[1:], columns=data[0])
df

COMPLETE: Data copied


,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11
...,...,...,...,...,...,...,...
4995,4996,73,993,330,2,debit,2017-03-30 13:47:17
4996,4997,48,789,234,2,cash,2017-03-16 20:36:16
4997,4998,56,867,351,3,cash,2017-03-19 5:42:42
4998,4999,60,825,354,2,credit_card,2017-03-16 14:51:18


On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis. 

**a.** Think about what could be going wrong with our calculation. Think about a better way to evaluate this data.

What we would expect AOV to be is less than $3145.13, since all shops are selling one model of sneakers. If the calculations are done right, then there must be an error in one or more order_amounts, which will be outliers in our calculations. In this case probably some orders that show an unexpectedly high per item price.  

**b.** What metric would you report for this dataset?

I would find the mean order_amount over the entire dataset, excluding the outliers. We will use the interquartile range on order amount per item to detect outliers and exclude them from our calculations.(It is also possible to use order_amount column to find outliers.)

In [80]:
import numpy as np

# Change datatype of necessary columns from object to integer
df['order_amount'] = df['order_amount'].astype(str).astype(int)
df['total_items'] = df['total_items'].astype(str).astype(int)

df['per_item_order_amount'] = df['order_amount'] / df['total_items']

q3, q1 = np.percentile(df['per_item_order_amount'], [75,25])
iqr = q3 - q1
ulimit = q3 + 1.5 * iqr

reduced_df = df[df['per_item_order_amount']<ulimit]

AOV = reduced_df['order_amount'].mean()

**c.** What is its value?

In [81]:
print("Average order value (AOV) is", AOV)

Average order value (AOV) is 300.1558229655313
